## Bibliotecas usadas

In [ ]:
import shutil
import json
import shapely
import shapefile
import shapely.wkb
import numpy as np
import psycopg2 as pg
# import pandas as pd
from pathlib import Path
from osgeo import ogr; ogr.UseExceptions()
from osgeo import osr; osr.UseExceptions()
from osgeo import ogr; ogr.UseExceptions()
from osgeo import osr; osr.UseExceptions()
from pathlib import Path
from json import dumps
from shapely.wkb import loads as wkb_loads

## Fazer alterações na tabela (Pandas)

In [ ]:
# Caso precise adicionar nome aos campos do csv e salvar um novo

# temp = pd.read_csv("temperatura_maxmin_GIS.csv", sep=";", header=None)
# temp.columns=['local', 'lat', 'lon', 'temp', 'min_max']
# temp.to_csv("temperatura_maxmin_GIS_header.csv",sep=";", index=False)

## Ler csv e salvar como shapefile (GDAL)

In [ ]:
#function: csv to shapefile

separator = ';'
csv_path = 'temperatura_maxmin_GIS.csv'
path_shp =  'result_GIS_shp/'
epsg = 4326

def save_csv_to_shapefile(path: Path, epsg: int, points_list, separator):
    if path.exists():
        shutil.rmtree(path) #clear directory
        
     # Create the output Driver   
    outDriver = ogr.GetDriverByName("ESRI Shapefile")
    # create the data source
    outDataSource = outDriver.CreateDataSource(str(path))
    # create the spatial reference, WGS84
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(epsg)
    # create the layer
    layer = outDataSource.CreateLayer("temperatura_maxmin", srs = srs, geom_type=ogr.wkbPoint)
    # Add the fields we're interested in
    layer.CreateField(ogr.FieldDefn("Nome", ogr.OFTString ))
    layer.CreateField(ogr.FieldDefn("MaxMin", ogr.OFTString ))
    layer.CreateField(ogr.FieldDefn("Temp", ogr.OFTReal))
    # Process the text file and add the attributes and features to the shapefile
    point_all = []
    
    for line in points_list:
        
        # split data 
        data = line.split(separator)
        # create the feature
        feature = ogr.Feature(layer.GetLayerDefn())
        # Set the attributes using the values from the delimited text file
        feature.SetField("Nome", data[0])
        feature.SetField("Temp", float(data[3]))
        feature.SetField("MaxMin", data[4])
        # create the WKT for the feature
        point = ogr.CreateGeometryFromWkt(shapely.geometry.Point(float(data[2]), float(data[1])).wkt)
        # Set the feature geometry using the point
        feature.SetGeometry(point)
        # Create the feature in the layer (shapefile)
        layer.CreateFeature(feature)
        point_all.append(point)
    # Save the data source    
    outDataSource.SyncToDisk()

In [ ]:
# read csv
with open(csv_path, 'r') as f:
    text_string = f.read()
    
points_list_ = [f for f in text_string.split('\n') if len(f.split(separator)) == 5]

# define path
save_path_shp = Path(path_shp)

# call function
save_csv_to_shapefile(save_path_shp, epsg, points_list_, separator)

## Ler o shapefile e salvar como geojson (pyshp)

In [ ]:
# function - option 1: shapefile to geojson 

shp_path = 'result_GIS_shp/temperatura_maxmin.shp'
geojson_path = 'shp_to_geojson/' 
 

def save_shp_to_geojson(shp_path: Path,  geojson_path: Path):
    if geojson_path.is_file():
        geojson_path.unlink() 
        
    # read the shapefile
    reader = shapefile.Reader(str(shp_path))
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]
    geom_all = []
    for sr in reader.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        geom_all.append(dict(type="Feature", geometry=geom, properties=atr)) 

        # write the GeoJSON file

    geojson = open(str(geojson_path), "w")
    geojson.write(dumps({"type": "FeatureCollection", "features": geom_all}, indent=2) + "\n")
    geojson.close()

# Ler o csv e salvar como geojson (GDAL)

In [ ]:
#function - option 2: csv to geojson

separator = ';'
csv_path = 'temperatura_maxmin_GIS.csv'
path_geojson_option2 = 'result_GIS_geojson/'

def save_csv_to_geojson(path: Path, points_list, separator):
    if path.is_file():
        path.unlink()

    # Create the output Driver  
    outDriver = ogr.GetDriverByName("GeoJSON")
    # Create the output GeoJSON
    outDataSource = outDriver.CreateDataSource(str(path))
    outLayer = outDataSource.CreateLayer("temperatura_maxmin.geojson", geom_type=ogr.wkbPoint)
    # Get the output Layer's Feature Definition
    featureDefn = outLayer.GetLayerDefn()
    # create a new feature
    outFeature = ogr.Feature(featureDefn)
    # Set new geometry  
    for line in points_list:
        
        data = line.split(separator)
        point = ogr.CreateGeometryFromWkt(shapely.geometry.Point(float(data[2]), float(data[1])).wkt)
        outFeature.SetGeometry(point)
        # Add new feature to output Layer
        outLayer.CreateFeature(outFeature)
    # Save the data source 
    outDataSource.SyncToDisk()

In [ ]:
# define path
save_geojson_path_option1 = Path(geojson_path_option1)/"temperatura_maxmin.json"

#call function
save_shp_to_geojson(shp_path, save_geojson_path_option1)


## PLUS: Ler o shapefile e salvar em um dicionário (GDAL)

In [ ]:
# PLUS function: read shapefile e get a dictionary (name and geom) w/ GDAL

def shapely_load(ogr_feat):
    return wkb_loads(ogr_feat.GetGeometryRef().ExportToWkb())

def shapefile_get_id(path):
    id_name = []
    geoms = []
    
    driver = ogr.GetDriverByName("ESRI Shapefile")
    dataSource = driver.Open(path)
    layer = dataSource.GetLayer()

    for feature in layer:
        
        geoms.append(shapely_load(feature))
        
        if feature.GetField("Nome"):
            id_name.append(feature.GetField("Nome"))  
        
        
    temp_dict = {'id_name':id_name, 'geom':geoms}
    
    return temp_dict

temp_dict = shapefile_get_id("result_GIS_shp/temperatura_maxmin.shp")


## Importar o geojson para o banco de dados

In [ ]:
#conectar com o servidor
def connection():
    conn = pg.connect( "host=host dbname='dbname' user=user password=password")
    return conn

# option 1 - abre multiplas conexões para inserir os dados, logo, demora mais
def insert_sql(insert_list, conn):
    
    for item in insert_list:
    
        local_name = item['nome']
        temp_max_min = item['maxmin']
        temp_value = item['temp']
        lon, lat = item['point']['coordinates']
    
    
        query = f"""
        INSERT INTO public.geojson_table (local, temp_max_min, geom, temp_value)
        VALUES ('{local_name}', '{temp_max_min}', 'SRID=4326;POINT({lon} {lat})', {temp_value});
        """
        
        conn = connection()
        cur = conn.cursor()

        cur.execute(query);
        conn.commit()

# option 2 - abre uma unica conexão para inserir os dados, logo, mais rapida
def insert_multiple_sql(insert_list, conn):
    query = f"""
        INSERT INTO public.geojson_table
        (local, temp_max_min, geom, temp_value)
        VALUES 
        """
    json_size = len(insert_list)-1
    for idx, item in enumerate(insert_list):
    
        local_name = item['nome']
        temp_max_min = item['maxmin']
        temp_value = item['temp']
        lon, lat = item['point']['coordinates']
        if idx != json_size:
            query += f"('{local_name}', '{temp_max_min}', 'SRID=4326;POINT({lon} {lat})', {temp_value}),"
        else:
            query += f"('{local_name}', '{temp_max_min}', 'SRID=4326;POINT({lon} {lat})', {temp_value});"

    conn = connection()
    cur = conn.cursor()

    cur.execute(query);
    conn.commit()

# criar tabela no banco e inserir dados
def insert_many(data):
    conn = connection()
    cur = conn.cursor()
    
    cur.execute("""DROP TABLE if exists public.json_table""");
    conn.commit()

    cur.execute("""CREATE TABLE public.json_table
              (
                  id serial NOT NULL PRIMARY KEY,
                  local char,
                  temp_max_min varchar(3),
                  geom geometry(Point, 4326),
                  temp_value real
              ) """)
    conn.commit()
    
#     insert_sql(insert_list, conn) #opção mais lenta
    insert_multiple_sql(insert_list, conn) #opção mais rápida
    
    cur.close()
    conn.close()
    
# converter todos os pontos em uma única geometria de linha     
def makeline():
    
    sql = '''
    SELECT ST_MakeLine(ARRAY(SELECT geom FROM public.geojson_table
    ORDER BY id ASC))
    '''
    conn = connection()
    cur = conn.cursor()

    cur.execute(sql,)

    data = None
    columns = [x.name for x in cur.description]
    for row in cur:
        data = dict(zip(columns, row))

    cur.close()
    conn.close()
    return data


insert_list = []

with open('shp_to_geojson/temperatura_maxmin.json') as geojson:
    data = json.load(geojson) 
    
    
    insert_list = []
    for obj in data['features']:
  
        geometry = obj['geometry']
        properties_origin = json.loads(json.dumps(obj['properties']))
        properties = dict((k.lower(), v) for k,v in properties_origin.items())
        
        # Feature
        obj['properties'] = properties
        
        # dic 
        properties['point'] = geometry

        # add list
        insert_list.append(properties)

# Chamar a função 
insert_many(data)
lines = makeline()
# print(lines)